In [1]:
import pandas as pd

df = pd.read_csv("../data/wrime.tsv", sep="\t")
df.columns.tolist()

['Sentence',
 'UserID',
 'Datetime',
 'Train/Dev/Test',
 'Writer_Joy',
 'Writer_Sadness',
 'Writer_Anticipation',
 'Writer_Surprise',
 'Writer_Anger',
 'Writer_Fear',
 'Writer_Disgust',
 'Writer_Trust',
 'Reader1_Joy',
 'Reader1_Sadness',
 'Reader1_Anticipation',
 'Reader1_Surprise',
 'Reader1_Anger',
 'Reader1_Fear',
 'Reader1_Disgust',
 'Reader1_Trust',
 'Reader2_Joy',
 'Reader2_Sadness',
 'Reader2_Anticipation',
 'Reader2_Surprise',
 'Reader2_Anger',
 'Reader2_Fear',
 'Reader2_Disgust',
 'Reader2_Trust',
 'Reader3_Joy',
 'Reader3_Sadness',
 'Reader3_Anticipation',
 'Reader3_Surprise',
 'Reader3_Anger',
 'Reader3_Fear',
 'Reader3_Disgust',
 'Reader3_Trust',
 'Avg. Readers_Joy',
 'Avg. Readers_Sadness',
 'Avg. Readers_Anticipation',
 'Avg. Readers_Surprise',
 'Avg. Readers_Anger',
 'Avg. Readers_Fear',
 'Avg. Readers_Disgust',
 'Avg. Readers_Trust']

In [2]:
df_writer = df.loc[:, "Writer_Joy":"Writer_Trust"]
df_reader1 = df.loc[:, "Reader1_Joy":"Reader1_Trust"]
df_reader2 = df.loc[:, "Reader2_Joy":"Reader2_Trust"]
df_reader3 = df.loc[:, "Reader3_Joy":"Reader3_Trust"]
df_avg_readers = df.loc[:, "Avg. Readers_Joy":"Avg. Readers_Trust"]

In [3]:
sentiments = [
    "Joy",
    "Sadness",
    "Anticipation",
    "Surprise",
    "Anger",
    "Fear",
    "Disgust",
    "Trust",
]

In [4]:
df_mean = pd.DataFrame(
    {
        "Writer": df_writer.mean().values,
        "Reader1": df_reader1.mean().values,
        "Reader2": df_reader2.mean().values,
        "Reader3": df_reader3.mean().values,
        "Avg. Readers": df_avg_readers.mean().values,
    },
    index=sentiments,
)
df_mean

,Writer,Reader1,Reader2,Reader3,Avg. Readers
Joy,0.793704,0.399792,0.433380,0.585069,0.464699
Sadness,0.642546,0.237199,0.462917,0.376412,0.351736
Anticipation,0.685764,0.418333,0.404190,0.589236,0.453634
Surprise,0.519375,0.377106,0.217407,0.419398,0.324120
Anger,0.234005,0.047037,0.057315,0.051389,0.049745
Fear,0.297546,0.316412,0.198542,0.341713,0.267639
Disgust,0.376389,0.191806,0.123241,0.411898,0.225972
Trust,0.381782,0.102338,0.050741,0.052685,0.049005


In [5]:
from scipy import stats

for s in sentiments:
    result_ttest = stats.ttest_rel(df[f"Writer_{s}"], df[f"Avg. Readers_{s}"])
    result_anova = stats.f_oneway(
        df[f"Reader1_{s}"],
        df[f"Reader2_{s}"],
        df[f"Reader3_{s}"],
    )

    print(f"[{s}]")
    print(f"Writer-Reader\t{result_ttest}")
    print(f"Reader1-2-3\t{result_anova}")
    print("")

df_writer_all = pd.concat([df[f"Writer_{s}"] for s in sentiments])

df_reader1_all = pd.concat([df[f"Reader1_{s}"] for s in sentiments])

df_reader2_all = pd.concat([df[f"Reader2_{s}"] for s in sentiments])

df_reader3_all = pd.concat([df[f"Reader3_{s}"] for s in sentiments])

df_readers_all = pd.concat([df[f"Avg. Readers_{s}"] for s in sentiments])

result_ttest_all = stats.ttest_rel(df_writer_all, df_readers_all)
result_anova_all = stats.f_oneway(
    df_reader1_all,
    df_reader2_all,
    df_reader3_all,
)

print("[All]")
print(f"Writer-Reader\t{result_ttest_all}")
print(f"Reader1-2-3\t{result_anova_all}")

[Joy]
Writer-Reader	Ttest_relResult(statistic=80.25003417829618, pvalue=0.0)
Reader1-2-3	F_onewayResult(statistic=554.539042468175, pvalue=1.5539080831810054e-240)

[Sadness]
Writer-Reader	Ttest_relResult(statistic=70.79990578608339, pvalue=0.0)
Reader1-2-3	F_onewayResult(statistic=975.2687039310326, pvalue=0.0)

[Anticipation]
Writer-Reader	Ttest_relResult(statistic=51.57500149339286, pvalue=0.0)
Reader1-2-3	F_onewayResult(statistic=643.7468914397273, pvalue=6.3662469253375824e-279)

[Surprise]
Writer-Reader	Ttest_relResult(statistic=47.59330935254756, pvalue=0.0)
Reader1-2-3	F_onewayResult(statistic=920.1743817944918, pvalue=0.0)

[Anger]
Writer-Reader	Ttest_relResult(statistic=63.14743423810323, pvalue=0.0)
Reader1-2-3	F_onewayResult(statistic=10.354422552031124, pvalue=3.187797096974353e-05)

[Fear]
Writer-Reader	Ttest_relResult(statistic=8.805663879488794, pvalue=1.3480061821061374e-18)
Reader1-2-3	F_onewayResult(statistic=563.1797375263448, pvalue=2.95737922915519e-244)

[Disgust

In [6]:
for s in sentiments:
    print("-" * 40, s, "-" * 40)
    df_s = df[
        [
            "Sentence",
            f"Writer_{s}",
            f"Reader1_{s}",
            f"Reader2_{s}",
            f"Reader3_{s}",
        ]
    ]
    df_list = [
        df_s.query(f"Reader1_{s} == Reader2_{s} == Reader3_{s} == {label}")
        for label in range(4)
    ]

    for i, df_s in enumerate(df_list):
        print(f"客観3人の {s} がすべて{i}\t{len(df_s)}")
    print(f"客観3人の {s} が等しい（合計）\t{sum([len(df_s) for df_s in df_list])}")
    print("")

    for i, df_s in enumerate(df_list):
        print(f"客観3人の {s} がすべて{i}")
        display(df_s.head())

---------------------------------------- Joy ----------------------------------------
客観3人の Joy がすべて0	26717
客観3人の Joy がすべて1	480
客観3人の Joy がすべて2	704
客観3人の Joy がすべて3	342
客観3人の Joy が等しい（合計）	28243

客観3人の Joy がすべて0


,Sentence,Writer_Joy,Reader1_Joy,Reader2_Joy,Reader3_Joy
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,0,0,0,0
2,早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。,1,0,0,0
3,眠い、眠れない。,0,0,0,0
5,表情筋が衰えてきてる｡まずいな…,0,0,0,0
7,おなかすいた…夜ご飯仲間募集｡,0,0,0,0


客観3人の Joy がすべて1


,Sentence,Writer_Joy,Reader1_Joy,Reader2_Joy,Reader3_Joy
159,はっ！こんな時間。でも慌てて眠らなくていいんだ！コロッケ食べてやる！,3,1,1,1
348,今日行った服屋さんで対応してくれた男の子、かなり乙女系で興味をそそられたので友達になってみた...,1,1,1,1
539,ドリンクは要らないという私にスイーツと水を差し出す前に、コーヒーはお好きですかと聞いてきた。...,3,1,1,1
1026,桃のフルーニュもｽﾁでしたねえ,0,1,1,1
1098,午前中まではマジでこの世の終わりみたいな体調だったけど15時くらいから復活した,1,1,1,1


客観3人の Joy がすべて2


...

,Sentence,Writer_Trust,Reader1_Trust,Reader2_Trust,Reader3_Trust
585,ハードル下げるってだいじ,1,1,1,1
1413,東海オンエアの面白動画の大多数はウンチが絡んでますけどもちろんウンチなしでも彼らは面白いので...,3,1,1,1
2386,傀と1番いい勝負したのは石川、次点で村田,0,1,1,1
3875,転職活動は短期決戦が絶対！\n3ヶ月以内に内定を取ってやるという意気込みがないと成功はしない,1,1,1,1
4023,久々にフレンチトースト作ったけど、やっぱりフランスパンで作るのが美味い。,0,1,1,1


客観3人の Trust がすべて2


,Sentence,Writer_Trust,Reader1_Trust,Reader2_Trust,Reader3_Trust
586,生きれててラッキーとも思うし生きて人生をなんとか紡ごうとしてる自分偉いやんとも思うし、もうち...,2,2,2,2
947,就寝前牛乳教教祖は適職だったんや…,0,2,2,2
3570,一番初めに買った車以外は日産です。,0,2,2,2
5050,新婚時に買った家電で、買い替えたのは6年目くらいで洗濯機と2～3年くらいでスチームオーブンレ...,0,2,2,2
32894,相手の好きなものに関心を寄せる\n\nやりきった日はビール\n疲れた日はカシオレ\nご飯のと...,2,2,2,2


客観3人の Trust がすべて3


,Sentence,Writer_Trust,Reader1_Trust,Reader2_Trust,Reader3_Trust
471,かなり年上の友人との晩餐。 顔を合わせたタイミングでごくたまに開催される年に数回のデート。 ...,3,3,3,3
557,久しぶりにskoop on somebodyを目にする機会があって、うわもうかっこえええええ...,3,3,3,3
961,札幌に旅行行くなら絶対いくべき ノースサファリ,0,3,3,3
1544,爆睡してたわ。どこで落ちたんだろ笑 私以上に疲れてるはずなのにずっと運転してくれるんだから、...,2,3,3,3
14854,友達と飲み会であった。いつも通り「辛いことあるけど楽しいこともあるから生きようぜ！」という結...,1,3,3,3


In [7]:
df.query(
    " and ".join([f"Reader1_{s} == Reader2_{s} == Reader3_{s}" for s in sentiments])
)

,Sentence,UserID,Datetime,Train/Dev/Test,Writer_Joy,Writer_Sadness,Writer_Anticipation,Writer_Surprise,Writer_Anger,Writer_Fear,...,Reader3_Disgust,Reader3_Trust,Avg. Readers_Joy,Avg. Readers_Sadness,Avg. Readers_Anticipation,Avg. Readers_Surprise,Avg. Readers_Anger,Avg. Readers_Fear,Avg. Readers_Disgust,Avg. Readers_Trust
2,早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。,1,2012/08/05 00:50,train,1,1,1,1,0,0,...,0,0,0,0,0,1,0,0,0,0
31,落語…どしよかな…それか映画いこかな。,1,2012/09/24 02:10,train,1,0,2,0,0,0,...,0,0,0,0,1,0,0,0,0,0
60,ほぼ毎日、パックのジュースをコンビニで買います。今日、袋の中にはジュースと割り箸が入ってまし...,1,2012/10/17 03:18,train,1,0,0,2,0,0,...,0,0,0,0,0,1,0,0,0,0
89,悲しくても辛くても、日々を生きなきゃいけなくて、そうしてうやむやにしながら生きていくんだなぁ。,1,2012/11/18 23:45,train,0,3,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
153,ひょう！激しいひょう！,1,2013/01/15 00:54,train,2,0,2,3,0,1,...,0,0,0,0,0,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43112,再チェックが4分の3まで完了。\n今日はココまで。\n\n…これで明日はカフェ＆サウナデーに...,80,2020/02/06 01:22,train,1,0,2,0,0,0,...,0,0,0,0,1,0,0,0,0,0
43144,突然、部屋の電気が消えた。\n\n停電ではない。原因不明。\n\nリラックスしてたので全然気...,80,2020/05/04 23:41,train,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
43151,望まないものをわんさとリコメンドして来るよりも、望むものにたどり着くのをサポートしてくれるシ...,80,2020/05/22 18:27,train,0,1,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0
43174,なぜか新潟に行きたい気分。\nタレカツ丼、食べたい…,80,2020/07/08 22:11,train,0,1,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
